# Fine-tuning BERT (and friends) for multi-label text classification

In this notebook, we are going to fine-tune BERT to predict one or more labels for a given piece of text. Note that this notebook illustrates how to fine-tune a bert-base-uncased model, but you can also fine-tune a RoBERTa, DeBERTa, DistilBERT, CANINE, ... checkpoint in the same way. 

All of those work in the same way: they add a linear layer on top of the base model, which is used to produce a tensor of shape (batch_size, num_labels), indicating the unnormalized scores for a number of labels for every example in the batch.



## Set-up environment

First, we install the libraries which we'll use: HuggingFace Transformers and Datasets.

In [55]:
%pip install -q transformers datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Note: you may need to restart the kernel to use updated packages.


## Load dataset

Next, let's download a multi-label text classification dataset from the [hub](https://huggingface.co/).

At the time of writing, I picked a random one as follows:   

* first, go to the "datasets" tab on huggingface.co
* next, select the "multi-label-classification" tag on the left as well as the the "1k<10k" tag (fo find a relatively small dataset).

Note that you can also easily load your local data (i.e. csv files, txt files, Parquet files, JSON, ...) as explained [here](https://huggingface.co/docs/datasets/loading.html#local-and-remote-files).



In [56]:
from datasets import load_dataset
from transformers import AutoTokenizer
import numpy as np
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
import pandas as pd
from ast import literal_eval    
from datetime import datetime
import datasets

In [57]:
level_selection = 'product'#'subcat'#'cat'
#dir(datasets)

In [58]:
df_train_raw = pd.read_csv('/kaggle/input/cleaned-toxic-comments/train_preprocessed.csv')

In [59]:
# less toxic gets stack on top of more toxic and less toxic gets a 0 target, more toxic is a 1 target
df_train = df_train_raw.copy()
df_train

,comment_text,id,identity_hate,insult,obscene,set,severe_toxic,threat,toxic,toxicity
0,explanation why the edits made under my userna...,0000997932d777bf,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
1,d aww he matches this background colour i m s...,000103f0d9cfb60f,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
2,hey man i m really not trying to edit war it...,000113f07ec002fd,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
3,more i can t make any real suggestions on im...,0001b41b1c6bb37e,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
4,you sir are my hero any chance you remember...,0001d958c54c6e35,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
159566,and for the second time of asking when your ...,ffe987279560d7ff,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
159567,you should be ashamed of yourself that is a ho...,ffea4adeee384e90,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
159568,spitzer umm theres no actual article for pros...,ffee36eab5c267c9,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0
159569,and it looks like it was actually you who put ...,fff125370e4aaaf3,0.0,0.0,0.0,train,0.0,0.0,0.0,0.0


In [60]:
df_train = df_train.sample(frac=1).reset_index(drop=True)
#df_train = df_train.iloc[:300000,:]# shrink down the training data so training doesn't take so long
df_val = df_train.iloc[:1000,:]
test_all = df_train.iloc[1000:10000,:]
df_train = df_train.iloc[10000:,:]


In [61]:
dataset = load_dataset("sem_eval_2018_task_1", "subtask5.english")

  0%|          | 0/3 [00:00<?, ?it/s]

In [62]:
df_train.columns

Index(['comment_text', 'id', 'identity_hate', 'insult', 'obscene', 'set',
       'severe_toxic', 'threat', 'toxic', 'toxicity'],
      dtype='object')

In [63]:
targets = ['identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic']
grouping = {k:v for k,v in zip(targets, range(len(targets)))}
train_dict = []
test_dict = []
val_dict = []
for d, f in zip([train_dict, test_dict, val_dict], [df_train, test_all, df_val]):
    for row in f.index:
        target_row = f[targets].loc[row].tolist()                       
        temp = {k:int(v) for k,v in zip(targets, target_row)}
        temp['text'] = f.comment_text[row]
        temp['ID'] = row
        d.append(temp)

In [64]:
print([len(i) for i in [train_dict, test_dict, val_dict]])

[149571, 9000, 1000]


In [65]:
target_row

[0.0, 0.0, 0.0, 1.0, 1.0, 1.0]

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [66]:
dir(dataset['train'])
dataset['train'].__class__
#https://arrow.apache.org/docs/python/generated/pyarrow.Table.html
import pyarrow as pa
test_table = pa.Table.from_pandas(pd.DataFrame(test_dict))
test_dataset = datasets.arrow_dataset.Dataset(test_table)

train_table = pa.Table.from_pandas(pd.DataFrame(train_dict))
train_dataset = datasets.arrow_dataset.Dataset(train_table)

val_table = pa.Table.from_pandas(pd.DataFrame(val_dict))
val_dataset = datasets.arrow_dataset.Dataset(val_table)


In [67]:
dataset.__class__
dataset = datasets.dataset_dict.DatasetDict({'test':test_dataset, 
                                             'train':train_dataset,
                                            'validation':val_dataset})


In [68]:
dataset['train']

Dataset({
    features: ['identity_hate', 'insult', 'obscene', 'severe_toxic', 'threat', 'toxic', 'text', 'ID'],
    num_rows: 149571
})

Let's check the first example of the training split:

In [69]:
example = train_dict[0]
example

{'identity_hate': 0,
 'insult': 1,
 'obscene': 1,
 'severe_toxic': 0,
 'threat': 0,
 'toxic': 1,
 'text': 'you  suck even if you stupid admins checkusers block us and find our ips  we ll always come back for more vandalism  trolling  harrassment  you stupid admins then cry over the fact of the enormous amounts of vandalism harrassment all day and all night because soon we will just take over and wikipedia wouldn t exist anymore  we will never stop vandalizing and harrassing other users like you on wikipedia to make their editing experience worse so they would quit ',
 'ID': 10000}

The dataset consists of tweets, labeled with one or more emotions. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [70]:
labels = [str(label) for label in grouping if label not in ['ID', 'text']]
id2label = {idx:label for idx, label in enumerate(labels)}
label2id = {label:idx for idx, label in enumerate(labels)}
labels[0]

'identity_hate'

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [71]:


tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128)
  # add labels
  labels_batch = {k: examples[k] for k in examples.keys() if k in labels}
  # create numpy array of shape (batch_size, num_labels)
  labels_matrix = np.zeros((len(text), len(labels)))
  # fill numpy array
  for idx, label in enumerate(labels):
    labels_matrix[:, idx] = labels_batch[label]

  encoding["labels"] = labels_matrix.tolist()
  
  return encoding

In [72]:
encoded_dataset = dataset.map(preprocess_data, batched=True, remove_columns=train_dataset.column_names)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [73]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [74]:
tokenizer.decode(example['input_ids'])

'[CLS] you suck even if you stupid admins checkusers block us and find our ips we ll always come back for more vandalism trolling harrassment you stupid admins then cry over the fact of the enormous amounts of vandalism harrassment all day and all night because soon we will just take over and wikipedia wouldn t exist anymore we will never stop vandalizing and harrassing other users like you on wikipedia to make their editing experience worse so they would quit [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [75]:
example['labels']

[0.0, 1.0, 1.0, 0.0, 0.0, 1.0]

In [76]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['insult', 'obscene', 'toxic']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [77]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [78]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

#model = torch.load('Colab_cats_model_202304131403').cuda()                                                           

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [79]:
batch_size = 16
metric_name = "f1"

In [80]:


args = TrainingArguments(
    f"bert-finetuned-sem_eval-english",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=2,#5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [81]:

# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [82]:
encoded_dataset['train'][0]['labels'].type()

'torch.FloatTensor'

In [83]:
encoded_dataset['train']['input_ids'][0]

tensor([  101,  2017, 11891,  2130,  2065,  2017,  5236,  4748, 21266,  4638,
        20330,  2015,  3796,  2149,  1998,  2424,  2256, 12997,  2015,  2057,
         2222,  2467,  2272,  2067,  2005,  2062,  3158,  9305,  2964, 18792,
         2075,  5292, 11335,  4757,  3672,  2017,  5236,  4748, 21266,  2059,
         5390,  2058,  1996,  2755,  1997,  1996,  8216,  8310,  1997,  3158,
         9305,  2964,  5292, 11335,  4757,  3672,  2035,  2154,  1998,  2035,
         2305,  2138,  2574,  2057,  2097,  2074,  2202,  2058,  1998, 16948,
         2876,  1056,  4839,  4902,  2057,  2097,  2196,  2644,  3158,  9305,
         6026,  1998,  5292, 11335, 18965,  2060,  5198,  2066,  2017,  2006,
        16948,  2000,  2191,  2037,  9260,  3325,  4788,  2061,  2027,  2052,
         8046,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [84]:
#forward pass
#outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train'][0]['labels'].unsqueeze(0))
#outputs

Let's start training!

In [85]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [86]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.039300,0.046577,0.764579,0.860463,0.918000
2,0.031800,0.048013,0.773389,0.878200,0.914000


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=9350, training_loss=0.039535691164394114, metrics={'train_runtime': 4113.7194, 'train_samples_per_second': 72.718, 'train_steps_per_second': 2.273, 'total_flos': 1.9677598514113536e+16, 'train_loss': 0.039535691164394114, 'epoch': 2.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [87]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.04801326245069504,
 'eval_f1': 0.7733887733887733,
 'eval_roc_auc': 0.8781998082849221,
 'eval_accuracy': 0.914,
 'eval_runtime': 5.2113,
 'eval_samples_per_second': 191.892,
 'eval_steps_per_second': 6.141,
 'epoch': 2.0}

## Inference

Let's test the model on a new sentence:

The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [90]:
torch.save(model,'Colab_model_{}'.format(datetime.now().strftime('%Y%m%d%H%M')))

In [91]:
text = test_all.narratives.tolist()[0]
re.split(r' |\:|\,|\.|\;|\n', text)

AttributeError: 'DataFrame' object has no attribute 'narratives'

In [ ]:
#test_all = df_val
import re
label_list = []
for text in test_all.narratives.tolist():
    
    encoding = tokenizer(text,  return_tensors="pt", padding=True, truncation=True,max_length=512, add_special_tokens = True)
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}
    #encoding = {i:torch.tensor([encoding[i][0][:512]]) for i in ['input_ids', 'token_type_ids', 'attention_mask']}

    outputs = model(**encoding)

    logits = outputs.logits
    logits.shape

    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= 0.5)] = 1
    # turn predicted id's into actual label names
    predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]
    label_list.append(predicted_labels)
    #print(predicted_labels, len(encoding['input_ids'][0]))

In [ ]:
#trainer.model()

In [ ]:
encoding

In [ ]:
output = trainer.predict(test_dataset)
encoding

In [ ]:
"""
model(encoding.input_ids.cuda(), encoding.attention_mask.cuda())
a=model(**{"input_ids":encoding.input_ids.cuda(), "token_type_ids":encoding.token_type_ids.cuda(), "attention_mask":encoding.attention_mask.cuda()})
b=model(**{k: v.to(trainer.model.device) for k,v in encoding.items()})
print(a, b)
"""

In [ ]:
# lookup table for code hierarchy level and column names, corresponding to the c_ and p_ variables below
level_coding_dict = {'cat' : {'coded_as' : 'ProdGroup_x'},
                     'subcat' : {'coded_as' : 'ProdSubGroup_x'},
                     'product' : {'coded_as' : 'CPSRMS_Prod_x'},
                    }

test_all['positive_'+level_selection] = label_list
test_results = test_all

In [ ]:
for lvl_selection in [level_selection]:#, 'subcat', 'product']:
    level_coding_dict[lvl_selection]['predicted'] = 'positive_' + lvl_selection
    report_lvl = level_coding_dict[lvl_selection]
    # use concise variables that are unlikely to be mixed up with other common variables
    p_ = report_lvl['predicted']
    c_ = report_lvl['coded_as']
    print(p_, c_)

    # aggregated predicted product codes are strings, need to convert them to integers
    # everywhere else they are integers, so this seems like the simplest way
    if lvl_selection == 'product': 
        newll = []
        for i in test_results[p_]:
            temp = []
            if i!= []:
                for j in i:
                    temp.append(int(j))
            newll.append(temp)    
        test_results[p_] = newll

    # list comprehension finds pos codes (FPs) from model not in the coded data
    test_results[p_ + '_FPs'] = [[k for k in i if k not in j] for i,j in zip(test_results[p_], test_results[c_])]
    # invert by taking out not for true positives TPs
    test_results[p_ + '_TPs'] = [[k for k in i if k in j] for i,j in zip(test_results[p_], test_results[c_])]
    # invert the FPs to FNs by switching the sublists j and i (coded as, predicted as)
    test_results[p_ + '_FNs'] = [[k for k in j if k not in i] for i,j in zip(test_results[p_], test_results[c_])]

In [ ]:
# turn product grouping and coding lookup table LUT into a dictionary
cats = codeLUT.Group.drop_duplicates().tolist()
subcats = codeLUT.SubGroup.drop_duplicates().tolist()
prods = codeLUT.Product.drop_duplicates().tolist()
# create dictionary in the format of {group : {subgroup : {product : code}}}
code_dict = {}
prodcode_dict = {}
for cat in cats:
    code_dict[cat] = {}
    temp = codeLUT[codeLUT.Group == cat]
    tempsubcats = temp.SubGroup.drop_duplicates().tolist()
    for subcat in tempsubcats:
        code_dict[cat][subcat] = {}
        subtemp = temp[temp.SubGroup == subcat]
        temp_prods = subtemp.Product.drop_duplicates().tolist()
        for prod in temp_prods:
            # {product as key : code as value}
            # code_dict[cat][subcat][prod] = subtemp[subtemp.Product == prod].Code.tolist()[0]
            # {code as key : product as value}
            code_dict[cat][subcat][subtemp[subtemp.Product == prod].Code.tolist()[0]] = prod
            prodcode_dict[subtemp[subtemp.Product == prod].Code.tolist()[0]] = prod

In [ ]:
precision_threshold = 0.95 # assume some final precision threshold above which the categories/subcats/products are automatically machine coded

for lvl_selection in [level_selection]:
    # set report_clf_lut aside to do precision analysis
    precision_df = test_results.copy()
    report_lvl = level_coding_dict[lvl_selection]
    # use concise variables that are unlikely to be mixed up with other common variables
    p_ = report_lvl['predicted']
    c_ = report_lvl['coded_as']
    print(p_, c_)

    # this is where we get overall precision, recall, and F1 scores for each cat/subcat/product
    # get precision for all elements in the category list (group, or subgroup, or product)
    # precision is TP / (TP + FP)
    precision_df
    all_pred_clf = []
    for item in test_results[p_].tolist():
        [all_pred_clf.append(element) for element in item if element not in all_pred_clf]

    precision_dictionary = {}
    recall_dictionary = {}
    F1_dictionary = {}
    # add the tp/fp/fn data to verify calculations
    TP_dictionary = {}
    FP_dictionary = {}
    FN_dictionary = {}
    TPFN_dictionary = {}
    for clf in all_pred_clf:
        #print(clf)
        if lvl_selection == 'product': clf = int(clf)
        # if the product is in or out of the list in the grid cell, it will return True or False
        tps = [clf in entry for entry in precision_df[p_ + '_TPs'].tolist()]
        fps = [clf in entry for entry in precision_df[p_ + '_FPs'].tolist()]
        fns = [clf in entry for entry in precision_df[p_ + '_FNs'].tolist()]
        tps = sum(tps)# sum of True values is like sum of 1's
        fps = sum(fps)
        fns = sum(fns)
        
        try: precision = tps/(tps+fps)
        except: precision = 0
        
        try: recall = tps/(tps+fns)# tps+fns is per report. some reports have the same code multiple times, but a classifier can only detect positive or negative, so tps+fns is different from total instances by a tiny amount in some cases
        except: recall = 0

        try: f1 = (2 * precision * recall) / (precision + recall)
        except: f1 = 0
        #print(precision)
        precision_dictionary[str(clf)] = precision
        recall_dictionary[str(clf)] = recall
        F1_dictionary[str(clf)] = f1
        
        TP_dictionary[str(clf)] = tps
        FP_dictionary[str(clf)] = fps
        FN_dictionary[str(clf)] = fns
        TPFN_dictionary[str(clf)] = tps + fns
        
    # turn the precision results into an output report
    # resulting in a table with as many rows as the level has (fewer for cats, more for subcats, more rows for prducts)
    # of category, machine code probability, and frequency of category occurrence
    # coerce to dataframe format
    # This is basically a merge operation, but instead of merge, it is sort and find the overlapping set
    param_dict = {}
    for dict_item, param in zip([precision_dictionary, recall_dictionary, F1_dictionary, TP_dictionary, FP_dictionary, FN_dictionary, TPFN_dictionary], ['precision', 'recall', 'F1', 'TPs', 'FPs', 'FNs','TPFN_sum']):
        param_dict[param] = list(dict_item.values())
    param_df = pd.DataFrame(param_dict)# turn dict to df
    param_df[lvl_selection] = list(dict_item.keys())# add code columns
    if lvl_selection == 'product': param_df[lvl_selection] = param_df[lvl_selection].astype(int)# change to compatable integer format
    valcounts = test_all.explode(c_)[c_].value_counts()# val counts per code
    valcounts = pd.DataFrame(valcounts.reset_index(level=0))# move code index to column
    valcounts.columns = [c_, 'frequency_coded_as']# columns named in reverse order, correct this
    precision_df = param_df.merge(valcounts, left_on=lvl_selection, right_on=c_, how='left')
    precision_df = precision_df.dropna()
    # sort so more precise and more frequent values are at the top
    precision_df = precision_df.sort_values(by=['precision', 'frequency_coded_as'], ascending=[False,False])
    #print(precision_series)
    # want to add one more column for the product level summary that converts code to name of code
    if lvl_selection == 'product':
        precision_df['product_name'] = [prodcode_dict[int(i)] for i in precision_df[c_].tolist()]
    
    # optional filtering by filter list, cats, subcats, and prods all in same list, loop iterates through levels, different batch of precision and recall per level
    #precision_df = precision_df[ pd.DataFrame(precision_df[c_].tolist()).isin(filter_list).any(1).values]
    #valcounts = valcounts[ pd.DataFrame(valcounts[c_].tolist()).isin(filter_list).any(1).values]
    
    
    # save table to csv
    precision_df.to_csv('model_evaluation_summary_{}_{}.csv'.format(c_, datetime.now().strftime('%Y%m%d%H%M')), index=False)
    #precision_series.head()
    # get number of records above a threshold
    t = precision_df[precision_df.precision >= precision_threshold]
    sum_ = (t.TPFN_sum * t.recall).sum()
    print("{} {} product instances are above the precision threshold out of {} instances, ratio of {}".format(str(round(sum_)),
                                                                                                   c_,
                                                                                                   valcounts.frequency_coded_as.sum(),#precision_df.TPFN_sum.sum()
                                                                                                   sum_/valcounts.frequency_coded_as.sum()
                                                                                                             ))  
    weighted_precision = []
    for row in range(precision_df.shape[0]):
        weighted_precision += [precision_df.precision.tolist()[row]] * precision_df.frequency_coded_as.astype(int).tolist()[row]
    print('Average weighted precision is', np.mean(weighted_precision))

print('Done.')